In [1]:
import re
import string
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize

from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
df = pd.read_csv("/content/SMSSpamCollection.txt", sep="\t", header=None)
print(df.shape)
# renaming the columns
df.rename(columns={0: 'label', 1: 'text'}, inplace=True)
df.head()

(5572, 2)


,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.isnull().sum()

label    0
text     0
dtype: int64

In [ ]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [3]:
# calculating length of each record in our df
df['length'] = df['text'].apply(lambda x: len(x))
df.head()

,label,text,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [4]:
# some text cleaning functions
def convert_to_lower(text):
    return text.lower()

def remove_numbers(text):
    number_pattern = r'\d+'
    without_number = re.sub(pattern=number_pattern, repl=" ", string=text)
    return without_number

def lemmatizing(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        lemma_word = lemmatizer.lemmatize(tokens[i])
        tokens[i] = lemma_word
    return " ".join(tokens)

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stopwords(text):
    removed = []
    stop_words = list(stopwords.words("english"))
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        if tokens[i] not in stop_words:
            removed.append(tokens[i])
    return " ".join(removed)

def remove_extra_white_spaces(text):
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    return without_sc

In [5]:
df['text'] = df['text'].apply(lambda x: convert_to_lower(x))
df['text'] = df['text'].apply(lambda x: remove_numbers(x))
df['text'] = df['text'].apply(lambda x: remove_punctuation(x))
df['text'] = df['text'].apply(lambda x: remove_stopwords(x))
df['text'] = df['text'].apply(lambda x: remove_extra_white_spaces(x))
df['text'] = df['text'].apply(lambda x: lemmatizing(x))

In [6]:
# calculating length of each record after performing text cleaning
df['length_after_cleaning'] = df['text'].apply(lambda x: len(x))
df.head()

,label,text,length,length_after_cleaning
0,ham,go jurong point crazy available bugis great wo...,111,78
1,ham,ok lar joking wif oni,29,21
2,spam,free entry wkly comp win fa cup final tkts st ...,155,101
3,ham,u dun say early hor c already say,49,33
4,ham,nah dont think go usf life around though,61,40


In [7]:
# converting string labels to int labels

label_map = {
    'ham': 0,
    'spam': 1,
}

df['label'] = df['label'].map(label_map)
df.head()

,label,text,length,length_after_cleaning
0,0,go jurong point crazy available bugis great wo...,111,78
1,0,ok lar joking wif oni,29,21
2,1,free entry wkly comp win fa cup final tkts st ...,155,101
3,0,u dun say early hor c already say,49,33
4,0,nah dont think go usf life around though,61,40


In [8]:
tf_without_balancing = TfidfVectorizer()
X_tf_wob = tf_without_balancing.fit_transform(df['text'])
X_tf_wob = X_tf_wob.toarray()

In [ ]:
print(X_tf_wob.shape)
print(X_tf_wob)

(5572, 7906)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
# splitting dataset
X_train_tf_wob, X_test_tf_wob, y_train_tf_wob, y_test_tf_wob = train_test_split(X_tf_wob, df['label'].values, test_size=0.2)

In [10]:
# initializing model
naiveBayes_wob = GaussianNB()

In [11]:
naiveBayes_wob.fit(X_train_tf_wob, y_train_tf_wob)

GaussianNB(priors=None, var_smoothing=1e-09)

In [12]:
y_pred_tf_wob = naiveBayes_wob.predict(X_test_tf_wob)

In [13]:
print(accuracy_score(y_test_tf_wob, y_pred_tf_wob))

0.8816143497757848


In [14]:
print(classification_report(y_test_tf_wob, y_pred_tf_wob))

              precision    recall  f1-score   support

           0       0.98      0.88      0.93       956
           1       0.55      0.89      0.68       159

    accuracy                           0.88      1115
   macro avg       0.77      0.88      0.80      1115
weighted avg       0.92      0.88      0.89      1115



####**with augmentation**

In [ ]:
!pip install transformers
!pip install nlpaug

In [16]:
import nlpaug.augmenter.word.context_word_embs as aug

In [ ]:
sample_text = df['text'].iloc[100]

In [ ]:
sample_text

'please dont text anymore nothing else say'

In [17]:
augmenter = aug.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
augmented_sample_text = augmenter.augment(sample_text)

In [ ]:
augmented_sample_text

'amy please dont text patrick anymore nothing else say'

In [ ]:
for i in range(5):
    print(augmenter.augment(sample_text))

please dont text anymore of nothing anywhere else say
please read dont text anymore how nothing else say
so please wait dont text anymore nothing else say
' please dont text anymore nothing wrong else say
paul please dont text jared anymore nothing else say


In [19]:
df['label'].value_counts()

0    4825
1     747
Name: label, dtype: int64

In [18]:
def augmentMyData(df, augmenter, repetitions=1, samples=200):
    augmented_texts = []
    # select only the minority class samples
    spam_df = df[df['label'] == 1].reset_index(drop=True) # removes unecessary index column
    for i in tqdm(np.random.randint(0, len(spam_df), samples)):
        # generating 'n_samples' augmented texts
        for _ in range(repetitions):
            augmented_text = augmenter.augment(spam_df['text'].iloc[i])
            augmented_texts.append(augmented_text)
    
    data = {
        'label': 1,
        'text': augmented_texts
    }
    aug_df = pd.DataFrame(data)
    df = shuffle(df.append(aug_df).reset_index(drop=True))
    return df

In [19]:
new_df = df.drop(labels=['length', 'length_after_cleaning'], axis=1)
new_df.head()

,label,text
0,0,go jurong point crazy available bugis great wo...
1,0,ok lar joking wif oni
2,1,free entry wkly comp win fa cup final tkts st ...
3,0,u dun say early hor c already say
4,0,nah dont think go usf life around though


In [60]:
aug_df = augmentMyData(new_df, augmenter, samples=2000)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [61]:
aug_df['label'].value_counts()

0    4825
1    2747
Name: label, dtype: int64

In [62]:
print("Original: ", df.shape)
print("Augmented: ", aug_df.shape)

Original:  (5572, 4)
Augmented:  (7572, 2)


In [63]:
df['label'].value_counts()

0    4825
1     747
Name: label, dtype: int64

In [24]:
# split the dataset before augmenting to avoid augmented data in valid set
X_train, _, y_train, _ = train_test_split(aug_df['text'], aug_df['label'].values, test_size=0.1)
_, X_test, _, y_test = train_test_split(df['text'], df['label'].values, test_size=0.5)

In [116]:
tf_with_aug = TfidfVectorizer()
X_train_tf = tf_with_aug.fit_transform(X_train)
X_train_tf = X_train_tf.toarray()

In [117]:
nb = GaussianNB()
nb.fit(X_train_tf, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [118]:
X_test_tf = tf_with_aug.transform(X_test)
X_test_tf = X_test_tf.toarray()

In [119]:
X_train_tf.shape, X_test_tf.shape

((6814, 8281), (2786, 8281))

In [120]:
y_preds = nb.predict(X_test_tf)

In [121]:
print(confusion_matrix(y_test, y_preds))
print(accuracy_score(y_test, y_preds))
print(classification_report(y_test, y_preds))

[[2048  356]
 [   0  382]]
0.8722182340272793
              precision    recall  f1-score   support

           0       1.00      0.85      0.92      2404
           1       0.52      1.00      0.68       382

    accuracy                           0.87      2786
   macro avg       0.76      0.93      0.80      2786
weighted avg       0.93      0.87      0.89      2786



####**RandomOverSampler**

In [20]:
!pip install imbalanced-learn

In [22]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

In [27]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'].values, test_size=0.25)

In [28]:
Counter(y_train)

Counter({0: 3612, 1: 567})

In [31]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [32]:
X_train_tf = vectorizer.transform(X_train)

In [35]:
X_train_tf = X_train_tf.toarray()
X_train_tf.shape

(4179, 6741)

In [42]:
X_test_tf = vectorizer.transform(X_test)
X_test_tf = X_test_tf.toarray()

In [37]:
overSampler = RandomOverSampler(0.5)
X_train_os, y_train_os = overSampler.fit_sample(X_train_tf, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [39]:
Counter(y_train_os)

Counter({0: 3612, 1: 1806})

In [41]:
nb = GaussianNB()
nb.fit(X_train_os, y_train_os)

GaussianNB(priors=None, var_smoothing=1e-09)

In [43]:
y_preds = nb.predict(X_test_tf)

In [44]:
print(confusion_matrix(y_test, y_preds))
print(accuracy_score(y_test, y_preds))
print(classification_report(y_test, y_preds))

[[1056  157]
 [  21  159]]
0.8722182340272793
              precision    recall  f1-score   support

           0       0.98      0.87      0.92      1213
           1       0.50      0.88      0.64       180

    accuracy                           0.87      1393
   macro avg       0.74      0.88      0.78      1393
weighted avg       0.92      0.87      0.89      1393

